In [7]:
import random

import numpy as np
import matplotlib.pyplot as plt


class Function:
    def __init__(self, value, delt=0.001):
        self.value = value

        def grad(x):
            array = []
            for i in range(len(x)):
                x[i] += delt
                first_val = value(x)
                x[i] -= 2 * delt
                second_val = value(x)
                x[i] += delt
                array.append((first_val - second_val) / (2 * delt))
            return np.array(array)

        self.grad = grad


wolfe_cond_template = lambda c1, c2, x, func, gk: lambda a, b: not (
            (func.value(x - ((a + b) / 2) * gk) <= (func.value(x) + c1 * ((a + b) / 2) * np.dot(gk, -gk))) and (
                np.dot(func.grad(x - ((a + b) / 2) * gk), -gk) >= c2 * np.dot(gk, -gk)))

wolfe_cond = lambda: ""


def grad_desc(func, lr, x, eps, is_wolfe=False):
    global wolfe_cond
    points = [x]

    while True:
        prev_x = x
        grad = func.grad(x)
        if is_wolfe:
            wolfe_cond = wolfe_cond_template(0.1, 0.9, x, func, grad)
        x = x - lr(Function(lambda a: func.value(x - a * grad)), is_wolfe) * grad
        points.append(x)
        if abs(func.value(x) - func.value(prev_x)) < eps:
            break

    return np.array(points)


def right_border_calc(func):
    right_start = 0.25
    zero = func.value(0)
    while zero >= func.value(right_start):
        right_start *= 2

    return right_start


def dichotomy(func, a_1, a_2, eps, delt, is_wolfe):
    cond = lambda a, b: abs(a - b) >= eps
    if is_wolfe:
        cond = wolfe_cond
    while cond(a_1, a_2):
        new_a_1 = (a_1 + a_2) / 2 - delt
        new_a_2 = (a_1 + a_2) / 2 + delt

        if func.value(new_a_2) > func.value(new_a_1):
            a_2 = new_a_2
        else:
            a_1 = new_a_1

    return (a_1 + a_2) / 2


def func_generator(n, k):
    vals = [random.uniform(1.0, k) for _ in range(n)]
    vals.sort()
    vals[0] = 1
    vals[n - 1] = k
    q, r = np.linalg.qr(np.random.rand(n, n))
    matr = np.matmul(np.matmul(q, np.diag(vals)), np.transpose(q))
    return matr


class GeneratedFunctionCalculator:
    def __init__(self, matr):
        self.matr = matr

        def calculate(vect):
            return sum(matr[i][j] * vect[i] * vect[j] for i in range(len(vect)) for j in range(len(vect)))

        self.calculate = calculate

In [8]:

def func(x):
    return (x[0] - 4) ** 2 / 10 + (5 - x[1]) ** 2


def lr_wrapper(eps, delt):
    return lambda func, is_wolfe: dichotomy(func, 0, right_border_calc(func), eps, delt, is_wolfe)

def example1(x):
    return (x[0] - 4) ** 2 / 10 + (5 - x[1]) ** 2
fc = GeneratedFunctionCalculator(func_generator(2, 5))
print(grad_desc(Function(fc.calculate), lr_wrapper(0.01, 0.0002),np.array((20., 20.)), 0.001, False))
print(grad_desc(Function(example1), lr_wrapper(0.01, 0.0001), np.array((20., 20.)), 0.0001, False))
print(grad_desc(Function(example1), lr_wrapper(0.01, 0.0001), np.array((20., 20.)), 0.0001, True))

[[2.00000000e+01 2.00000000e+01]
 [7.56230200e+00 1.85407915e+01]
 [8.36750861e+00 7.87913081e+00]
 [3.24277528e+00 7.55508898e+00]
 [3.28767619e+00 2.89105133e+00]
 [1.18271981e+00 2.86921584e+00]
 [1.28514517e+00 1.22889518e+00]
 [4.59302158e-01 1.16586510e+00]
 [5.33279988e-01 5.30827689e-01]
 [2.00480215e-01 4.93259275e-01]
 [2.23162892e-01 2.09071393e-01]
 [8.60093208e-02 2.00978558e-01]
 [8.77045398e-02 7.66905608e-02]
 [3.13576895e-02 7.63313114e-02]
 [3.42730586e-02 3.26106307e-02]
 [1.21721118e-02 3.10183944e-02]
 [1.42196366e-02 1.40886424e-02]]
[[20.         20.        ]
 [18.38734125  4.88132422]
 [ 4.82040989  6.0004095 ]
 [ 4.73771982  4.99208504]
 [ 4.0685785   5.06387687]
 [ 4.06166639  4.99949462]
 [ 4.0035164   5.0042602 ]
 [ 4.00316198  4.99996629]]
[[20.         20.        ]
 [18.39984     4.9985    ]
 [-4.639904    5.0225    ]
 [ 9.1839424   4.6625    ]
 [ 7.11036544  6.01249999]
 [ 6.79929779  4.99989875]
 [ 2.32042132  5.00151875]
 [ 5.00774721  4.97721875]
 [ 4.